# Convolutional Neural Network Example

In [29]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as utils
import matplotlib as plt
from torch.autograd import Variable
import pandas as pd
import numpy 

In [5]:
# Test access to gpu/cuda
torch.cuda.is_available()

True

### Load the MNIST Data

In [6]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [7]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

In [8]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

### Build a two-layer Convolutional Neural Network

In [9]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
cnn = CNN()
cnn.cuda()

CNN (
  (layer1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (fc): Linear (1568 -> 10)
)

In [11]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

### Model Training

In [12]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Iter [100/600] Loss: 0.1706
Epoch [1/5], Iter [200/600] Loss: 0.1483
Epoch [1/5], Iter [300/600] Loss: 0.1638
Epoch [1/5], Iter [400/600] Loss: 0.0580
Epoch [1/5], Iter [500/600] Loss: 0.0429
Epoch [1/5], Iter [600/600] Loss: 0.0445
Epoch [2/5], Iter [100/600] Loss: 0.0171
Epoch [2/5], Iter [200/600] Loss: 0.0532
Epoch [2/5], Iter [300/600] Loss: 0.0326
Epoch [2/5], Iter [400/600] Loss: 0.0609
Epoch [2/5], Iter [500/600] Loss: 0.0344
Epoch [2/5], Iter [600/600] Loss: 0.0429
Epoch [3/5], Iter [100/600] Loss: 0.0238
Epoch [3/5], Iter [200/600] Loss: 0.0587
Epoch [3/5], Iter [300/600] Loss: 0.0232
Epoch [3/5], Iter [400/600] Loss: 0.0525
Epoch [3/5], Iter [500/600] Loss: 0.0145
Epoch [3/5], Iter [600/600] Loss: 0.0275
Epoch [4/5], Iter [100/600] Loss: 0.0035
Epoch [4/5], Iter [200/600] Loss: 0.0142
Epoch [4/5], Iter [300/600] Loss: 0.0188
Epoch [4/5], Iter [400/600] Loss: 0.0154
Epoch [4/5], Iter [500/600] Loss: 0.0070
Epoch [4/5], Iter [600/600] Loss: 0.0450
Epoch [5/5], Ite

## Inference (Test the model)

In [13]:
my_labels = []
my_predictions = []

# Test the Model
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    my_labels += labels.tolist()
    my_predictions += predicted.cpu().tolist()

In [40]:
print('Test Accuracy of the model on the 10000 test images: %d%%\n' % (100 * correct / total))
print('%d out of %d classified correctly' % (correct, total))
# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

Test Accuracy of the model on the 10000 test images: 98%

9898 out of 10000 classified correctly


In [36]:
df = pd.DataFrame({})
df['label'] = my_labels
df['prediction'] = my_predictions
df[df['label'] != df['prediction']].head(10)

,label,prediction
92,9,4
259,6,0
321,2,7
340,5,3
359,9,4
445,6,0
449,3,5
582,8,3
659,2,1
674,5,3


In [16]:
type(images.cpu()[0].data)

torch.FloatTensor

In [17]:
to_pil = transforms.ToPILImage()
img = to_pil(images.cpu()[0].data)

In [20]:
img

In [33]:
def imshow2(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [34]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [35]:
# show images
imshow2(utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

AttributeError: module 'matplotlib' has no attribute 'imshow'